- Nutzung verschiedener Residual Layers
- Skip Layer
- Nutzung nur des B(3,3)-Block
- Mit der tiefe des KNN: Overfitting, freie Parameter, lange Trainingsdauer
--> Verbreiterung des Netzwerks anstatt dieses tiefer zu machen. 
--> Feature wurden mit k multipliziert --> Anzahl der Kernel pro Dimension nimmt zu. 
--> 40 = 40 B(3,3)-Blöcke hintereinander geschaltet. 
- Anzahl der freien Parameter bleibt gleich. 
- Dropout: Zufällig werden einzelne Werte in den Layern = 0 gesetzt --> kann übertraining verbessern
--> Kann man vernachlässigen. Weil es kaum einen Unterschied gemacht hat. 
- WRN: Wide Residual Network
- 22: Tiefe
- nächste Zahl: widering Faktor: k (Mit denen die Anzahl der Kernel multipliziert wird)
- Es soll eine Variante genommen werden. 
--> Trainieren des Modells für Vögel und Flugzeuge
- Schauen auf welche Genauigkeiten man kommt. 

### Hausarbeit 8 

• Modifizieren Sie das ResNet Beispiel aus dem Buch so, dass es dem 
Netzwerk aus dem Paper “Wide Residual Networks” von Zagoruyko et al. 
entspricht <br>
• Verwenden Sie die gleichen Augmentation Methods wie in dem Paper 
(RandomHorizontalFlip und RandomCrop)<br>
• Vergleichen Sie die Accuracy (Erkennungsraten) für Trainings- und 
Validierungsdaten mit dem Beispiel aus dem Buch<br>
• Das Netzwerk dabei nur für die Klassifikation von Vögeln und Flugzeugen 
trainieren.<br>
• Verwenden Sie 28 B(3,3) Blöcke mit k=2, [WRN-28-2-B(3,3)]<br>

#### 1.Schritt: Laden der notwendigen Bibliotheken

In [4]:
# Laden der notwendigen Bibliotheken
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import os
import collections
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import datetime 
import torch
import time
import random

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

#### 2.Schritt: Laden der Bilddaten aus dem CIFAR10-Datensatz. Klassen: Vögel und Flugzeuge

In [5]:
# Definition der einzelnen Klassen
class_names_bird_plane = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

# Laden des Datensatzes: Umwandlung in einen Tensor und Normalisierung der Daten
data_path = '../data-unversioned/p1ch6/'

# Trainingsdatensatz
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

# Validierungsdatensatz
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

# Bildung eines neuen Datensatzes, nur mit Bird und Flugzeug Bildern
label_map = {0: 0, 2: 1}
class_names_bird_plane = ['airplane', 'bird']

# Trainingsdatensatz
cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]

# Validierungsdatensatz
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

Files already downloaded and verified
Files already downloaded and verified


#### 3.Schritt: Definition des ursprünglichen Modells mit dem 3x3-Kernel:


In [6]:
# Definition des Baseline-Modells:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Definition eines Convolutional-Layers mit einem 3x3 Kernel
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out
    
# Definition des Trainingloops
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):  
        loss_train = 0.0
        for imgs, labels in train_loader:  
            outputs = model(imgs)  
            loss = loss_fn(outputs, labels)  
            optimizer.zero_grad()  
            loss.backward()  
            optimizer.step()  
            loss_train += loss.item()  

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader))) 
            
# Trainieren des Modells
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True) 

model3x3 = Net()  
optimizer = optim.SGD(model3x3.parameters(), lr=1e-2)  
loss_fn = nn.CrossEntropyLoss()  

training_loop(  
    n_epochs = 10,
    optimizer = optimizer,
    model = model3x3,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

# Berechnung der Genauigkeit des Modells für die Trainings und Validierungsdaten:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)
all_acc_dict = collections.OrderedDict()

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():  
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0] 
                correct += int((predicted == labels).sum())  

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model3x3, train_loader, val_loader)

2023-06-09 20:49:18.079073 Epoch 1, Training loss 0.5803694548500571
2023-06-09 20:49:38.944224 Epoch 10, Training loss 0.33222396576860147
Accuracy train: 0.85
Accuracy val: 0.84
